In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [5]:

import os
import random
import torch
import timm
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader


In [6]:
device = torch.device("cpu")
print("Using device:", device)



Using device: cpu


In [7]:
print(torch.__version__)
print(timm.__version__)

2.6.0+cu124
1.0.15


In [8]:
base_path = "/kaggle/input/plantdisease/PlantVillage"

# Check the folders (should be class names like Tomato___Early_blight etc.)
print("Classes available in the dataset:")
print(os.listdir(base_path))


Classes available in the dataset:
['Pepper__bell___Bacterial_spot', 'Potato___healthy', 'Tomato_Leaf_Mold', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Bacterial_spot', 'Tomato_Septoria_leaf_spot', 'Tomato_healthy', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato_Early_blight', 'Tomato__Target_Spot', 'Pepper__bell___healthy', 'Potato___Late_blight', 'Tomato_Late_blight', 'Potato___Early_blight', 'Tomato__Tomato_mosaic_virus']


In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

full_dataset = datasets.ImageFolder(base_path, transform=transform)
print("Total images:", len(full_dataset))
print("All class names:", full_dataset.classes)


Total images: 20638
All class names: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [10]:
tomato_class_names = [cls for cls in full_dataset.classes if "Tomato" in cls]
print("Tomato classes:", tomato_class_names)

Tomato classes: ['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [11]:
tomato_indices = [i for i, (path, label) in enumerate(full_dataset.samples)
                  if full_dataset.classes[label] in tomato_class_names]

# Subset only tomato images
tomato_dataset = Subset(full_dataset, tomato_indices)
print("Total tomato images:", len(tomato_dataset))

Total tomato images: 16011


In [12]:
#Shuffle and split (1/3 train, 2/3 test)
random.shuffle(tomato_indices)
train_size = int(len(tomato_indices) * 1/3)
test_size = len(tomato_indices) - train_size

train_indices = tomato_indices[:train_size]
test_indices = tomato_indices[train_size:]

train_set = Subset(full_dataset, train_indices)
test_set = Subset(full_dataset, test_indices)

print("Train size:", len(train_set))
print("Test size:", len(test_set))


Train size: 5337
Test size: 10674


In [13]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

In [14]:
model = timm.create_model(
    'vit_base_patch16_224',
    pretrained=True,
    num_classes=len(tomato_class_names)
)

model = model.to(device)
print("ViT model loaded on:", device)


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

ViT model loaded on: cpu


In [15]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [16]:
# Map original label to new label index for tomato classes
label_map = {full_dataset.class_to_idx[cls]: i for i, cls in enumerate(tomato_class_names)}

epochs = 1

for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        mapped_labels = torch.tensor([label_map[l.item()] for l in labels])
        images = images.to(device)
        labels = mapped_labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Print progress every 10 batches
        if batch_idx % 10 == 0:
            print(f"Epoch {epoch+1} Batch {batch_idx}/{len(train_loader)} - Loss: {loss.item():.4f}", flush=True)

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}] completed | Loss: {running_loss:.4f} | Accuracy: {acc:.2f}%")


Epoch 1 Batch 0/167 - Loss: 2.2625
Epoch 1 Batch 10/167 - Loss: 1.8365
Epoch 1 Batch 20/167 - Loss: 1.1931
Epoch 1 Batch 30/167 - Loss: 0.4527
Epoch 1 Batch 40/167 - Loss: 0.9486
Epoch 1 Batch 50/167 - Loss: 0.5839
Epoch 1 Batch 60/167 - Loss: 0.3020
Epoch 1 Batch 70/167 - Loss: 0.2071
Epoch 1 Batch 80/167 - Loss: 0.2044
Epoch 1 Batch 90/167 - Loss: 0.4133
Epoch 1 Batch 100/167 - Loss: 0.2975
Epoch 1 Batch 110/167 - Loss: 0.1033
Epoch 1 Batch 120/167 - Loss: 0.0266
Epoch 1 Batch 130/167 - Loss: 0.2380
Epoch 1 Batch 140/167 - Loss: 0.5613
Epoch 1 Batch 150/167 - Loss: 0.3672
Epoch 1 Batch 160/167 - Loss: 0.2622
Epoch [1/1] completed | Loss: 101.9867 | Accuracy: 79.88%


In [17]:
# Evaluate on the test set
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        mapped_labels = torch.tensor([label_map[l.item()] for l in labels])
        images = images.to(device)
        labels = mapped_labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc = 100 * correct / total
print(f"Test Accuracy: {test_acc:.2f}%")


Test Accuracy: 91.70%


In [19]:
# Save the trained model
save_path = "/kaggle/working/plantmd_vit_tomato.pth"
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")


Model saved to /kaggle/working/plantmd_vit_tomato.pth
